## ReLCo

In [ ]:
import quartz
from quartz import layers
import numpy as np
import torch
import torch.nn as nn
t_max = 2**8
dims = (1,1,1)
n_outputs = 1

np.random.seed(seed=47)
weights = np.array([[-1.]])
biases = (np.random.rand(n_outputs) - 0.5) / 2

loihi_model = quartz.Network([
    layers.InputLayer(dims=dims),
    layers.Dense(weights=weights, biases=None),
    layers.MonitorLayer(),
])

relco_probe = quartz.probe(loihi_model.layers[1].blocks[0])
neuron_probes = [quartz.probe(neuron) for neuron in loihi_model.layers[1].blocks[0].neurons]

values = np.random.rand(np.product(dims)) # np.ones((np.product(dims))) * 0.5
values = np.array([0.5])
values = np.array([1])
quantized_values = (values*t_max).round()/t_max
quantized_biases = (biases*t_max).round()/t_max

pt_model = nn.Sequential(
    nn.Linear(in_features=np.product(dims), out_features=n_outputs), 
    nn.ReLU()
)
pt_model[0].weight = torch.nn.Parameter(torch.tensor(weights))
pt_model[0].bias = torch.nn.Parameter(torch.tensor(np.zeros_like(quantized_biases)))
model_output = pt_model(torch.tensor(quantized_values)).detach().numpy()
model_output

In [ ]:
loihi_model(values, t_max)

In [ ]:
relco_probe.output()[1]

In [ ]:
weights


In [ ]:
[neuron.plot() for neuron in neuron_probes]

In [ ]:
calc_probe.plot()

## Max pooling

In [ ]:
%matplotlib inline
import quartz
#import numpy as np
recall_spike_times = [[1],[10],[20],]
t_max = 2**8
run_time = 4*t_max

constant_monitor = False

values = [0, 0.152, 0.245]
inputs = []
constants = []
for i, value in enumerate(values):
    constants.append(quartz.blocks.ConstantDelay(value, name="const"+str(i)+":", promoted=True, monitor=constant_monitor))
    inputs.append(quartz.blocks.Splitter(number=i, monitor=False))

pool = quartz.blocks.MaxPooling(inputs, split_input=False, split_output=False, name="", monitor=True)
pool.blocks += constants
pool.blocks += inputs

weight_e, weight_acc, t_syn, t_min, t_neu = pool.get_params_at_once()
for const, inp in zip(constants, inputs):
    const.output_neurons()[0].connect_to(inp.input_neurons()[0], weight_e, t_syn)

pool.run_on_loihi(run_time, recall_spike_list=recall_spike_times, t_max=t_max, partition='loihi', probe_selection=["input", "sync",])